In [1]:
#!/usr/bin/env python
################################################################
#  Copyright (C) 2015 OpenEye Scientific Software, Inc.
################################################################
from __future__ import print_function
import os, sys
import numpy as np
from simtk import openmm
import base64, pickle
import parmed
import openeye.oechem as oechem
from simtk import unit
from simtk.openmm import app

The OEB file should contain a molecule with a saved state containing Positions, Velocities, and Parameters from the simulation.
Reference: https://github.com/openeye-private/openmm_orion/blob/master/OpenMMCubes/cubes.py#L353

In [2]:
# Read back in the OEB file
infile = '../examples/data/9PC1X-simulation.oeb.gz'
#infile = 'WNE6S-warmup.oeb.gz'
ifs = oechem.oemolistream(infile)
mol = oechem.OEMol()
oechem.OEReadMolecule(ifs, mol)

True

All our objects are stored as generic tagged data. We have to regenerate the topology from the encoded Parmed Structure dictionary by decoding the dictionary and loading it into an empty Parmed Structure.

In [3]:
struct_tag = oechem.OEGetTag('structure')
encoded_structure = mol.GetData(struct_tag)
decoded_structure = base64.b64decode(encoded_structure)
struct_dict = pickle.loads(decoded_structure)
struct = parmed.structure.Structure()
struct.__setstate__(struct_dict)
topology = struct.topology
topology

<Topology; 3 chains, 9964 residues, 31994 atoms, 22215 bonds>

Likewise, we have to decode the OpenMM System to regenerate the Simulation object

In [4]:
sys_tag = oechem.OEGetTag('system')
encoded_system = mol.GetData(sys_tag)
system = openmm.XmlSerializer.deserialize(encoded_system)
system

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7fbc9d3b2b40> >

In [5]:
# Now regenerate the simulation object.
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 1/unit.picoseconds, 0.002*unit.picoseconds)
simulation = app.Simulation(topology, system, integrator)

This Simulation object should have no positions/velocities stored since we've created a new Simulation for restarting.

In [8]:
sim_state = simulation.context.getState(getPositions=True,getVelocities=True,getParameters=True)
sim_vel = sim_state.getVelocities(asNumpy=True) 
not np.any(sim_vel)

True

Since we want to restart the simulation, we grab the saved state containing the positions, velocities and parameters from the tagged data. Then, set the state from the current context to the saved state.

In [10]:
encoded_state = mol.GetData(oechem.OEGetTag('state'))
state = openmm.XmlSerializer.deserialize(encoded_state)
saved_vel = state.getVelocities()
not np.any(saved_vel)

False

Now, the context should have it's velocities set from the saved state.

In [11]:
simulation.context.setState(state)
sim_state = simulation.context.getState(getPositions=True,getVelocities=True,getParameters=True)
sim_vel = sim_state.getVelocities()
not np.any(sim_vel)

False

In [12]:
sim_vel == saved_vel

True